Библиотеки

In [1]:
# Импорт PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Импорт NumPy
import numpy as np

# Импорт Matplotlib
import matplotlib.pyplot as plt

# Импорт Librosa
import librosa
import librosa.display

#импорт os
import os

пути для входных и целевых данных

In [2]:
mix_dir = "C:\для учебы\Диплом\Диплом маг\DataSet\SDataSet\mix"
inst_dir = "C:\для учебы\Диплом\Диплом маг\DataSet\SDataSet\instruments"

In [ ]:
mix_list = os.listdir(mix_dir)


100